## TODOs
* in compose_tfms2 return a custom type
* this custom type should handle ToTensor

In [ ]:
from fastai2.basics import *
from fastai2.text.all import *
from pigboat import *

In [ ]:
from snorkel.labeling.model import *

In [ ]:
source = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
df = pd.read_csv(source/'texts.csv')

In [ ]:
tkzer = Tokenizer.from_df('text')
tkzer.setup(df)

In [ ]:
class Tokens(L): pass

In [ ]:
@patch
def __call__(self:Tokenizer, x, **kwargs):
    return retain_type(super(Tokenizer, self).__call__(x, **kwargs), typ=Tokens)

In [ ]:
# Cats = create_categories('pos neg')

In [ ]:
POS,NEG = 'positive','negative'
vocab = register_categories(POS,NEG)

In [ ]:
def WordLabeller(words, label):
    def _inner(x:str):
        for w in L(words):
            if w in x: return label
    return TypeDispatch2([_inner])

In [ ]:
POS_WORDS  = ['enjoy', 'liked', 'love', 'masterpiece', 'enjoyed', 'loved', 'nice', 'good acting']
NEG_WORDS = ['hate', 'poor', 'bad', 'hated', 'dislike', 'disliked', 'suck', 'lame', 'worst']

In [ ]:
lblr = []
lblr += [WordLabeller(w, POS) for w in POS_WORDS]
lblr += [WordLabeller(w, NEG) for w in NEG_WORDS]

In [ ]:
pipe_items = Pipeline([tkzer])

In [ ]:
pipe = Pipeline2([tkzer], lblr)

In [ ]:
lbls_true = TfmdLists(df, [ColReader('label'), vocab.o2i.get, lambda o: o-1])
lbls_true = np.array(list(lbls_true))

In [ ]:
class TensorNoisyLabels(TensorBase): pass

In [ ]:
class NoisyLabels(L):
    @classmethod
    def create(cls, x):
        return cls(x)

In [ ]:
@ToTensor
def encodes(self, o:NoisyLabels): return tensor(o)

In [ ]:
tls = TfmdLists(df, [ColReader('text'), pipe])
tls_items = TfmdLists(df, [ColReader('text'), pipe_items])

In [ ]:
dls = tls.dataloaders(verbose=True)

Setting up after_item: Pipeline: 
Setting up before_batch: Pipeline: 
Setting up after_batch: Pipeline: 


In [ ]:
b = dls.one_batch()

In [ ]:
b

TaskLabels([[0, 0, 2,  ..., 0, 0, 0],
        [0, 0, 2,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 2,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0')

In [ ]:
# How to apply in parellel
lbls = tensor(list(tls))

In [ ]:
# lbls[lbls==vocab.o2i[ABSTAIN]] = -1
# Only works because ABSTAIN is already 0
lbls -= 1

In [ ]:
lbls_arr = np.array(lbls)

In [ ]:
lbl_m = LabelModel(2, verbose=True)
lbl_m.fit(lbls_arr, n_epochs=500, log_freq=100)

In [ ]:
lbl_m.score(lbls_arr, lbls_true)

{'accuracy': 0.6590214067278287}

In [ ]:
dls = tls.dataloaders(shuffe=False)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b)

3

In [ ]:
ToTensor()(items[0])

TensorCategory(0)